# The Binance API

## Introduction

In the previous Episodes we saw how could we obtain data from Eikon using the Python API to connect to its Terminal. We saw it was very easy and straightforward to obtain a constant stream of information or the historic data for a particular financial instrument, using an app key and the API methods.

I spent a lot of nice words praising the ease of connection and the simplicity of the API’s design. The time has come to see the flip of the coin. I will introduce it by telling you a story.

In [10]:
import requests
from datetime import datetime

# ping the api
# response is an empy dictionary
response = requests.get("https://api.binance.com/api/v3/ping")
print(response.json())

# get the time of the server
response = requests.get("https://api.binance.com/api/v3/time")
print(response.json())

# server time is in milliseconds, fromtimestamps expects
# seconds: divide it by 1000 to get seconds.
server_timestamp = response.json().get('serverTime')
server_time = datetime.fromtimestamp(int(server_timestamp/1000))
print(f"The server time is: {server_time}")

{}
{'serverTime': 1650381345344}
The server time is: 2022-04-19 17:15:45


In [ ]:
import json

# get the info of all the exchange
# do not print all the response! Too messy
response = requests.get("https://api.binance.com/api/v3/exchangeInfo")

# get the info of a particular symbol
response = requests.get("https://api.binance.com/api/v3/exchangeInfo?symbol=BNBBTC")

# or this (to avoid long strings)
params = dict(symbol="BNBBTC")
response = requests.get("https://api.binance.com/api/v3/exchangeInfo",params=params)

print(json.dumps(response.json(), indent=2))


In [19]:
# get the info of a list of symbols
response = requests.get('https://api.binance.com/api/v3/exchangeInfo?symbols=["BNBBTC","BTCUSDT"]')

# or this (to avoid long strings)
params = dict(symbols='["BNBBTC","BTCUSDT"]')
response = requests.get("https://api.binance.com/api/v3/exchangeInfo",params=params)

print(json.dumps(response.json(), indent=2))


{
  "timezone": "UTC",
  "serverTime": 1650382234591,
  "rateLimits": [
    {
      "rateLimitType": "REQUEST_WEIGHT",
      "interval": "MINUTE",
      "intervalNum": 1,
      "limit": 1200
    },
    {
      "rateLimitType": "ORDERS",
      "interval": "SECOND",
      "intervalNum": 10,
      "limit": 50
    },
    {
      "rateLimitType": "ORDERS",
      "interval": "DAY",
      "intervalNum": 1,
      "limit": 160000
    },
    {
      "rateLimitType": "RAW_REQUESTS",
      "interval": "MINUTE",
      "intervalNum": 5,
      "limit": 6100
    }
  ],
  "exchangeFilters": [],
  "symbols": [
    {
      "symbol": "BNBBTC",
      "status": "TRADING",
      "baseAsset": "BNB",
      "baseAssetPrecision": 8,
      "quoteAsset": "BTC",
      "quotePrecision": 8,
      "quoteAssetPrecision": 8,
      "baseCommissionPrecision": 8,
      "quoteCommissionPrecision": 8,
      "orderTypes": [
        "LIMIT",
        "LIMIT_MAKER",
        "MARKET",
        "STOP_LOSS_LIMIT",
        "TAKE_PROF

### #2 making an http request 

We will use the python library ```requests``` and send an http request to localhost 9060 and route api/status and look at the response.

In [1]:
import requests

# check if eikon application is opened
try:
    # send a request to port 9060
    # returns a status 'ST_PROXY_READY' if eikon application is opened
    # throws a timeout error if the application is closed
    request = requests.get('http://localhost:9060/api/status')
    response = request.json()

    if response['statusCode'] == 'ST_PROXY_READY': 
        print('The Eikon proxy is up and running')
except:
    print('It seems your Eikon Terminal is not running')
    # take action here

The Eikon proxy is up and running


In this case, the request throws a timeout error if the application is not running and we need to wrap our request into a try-except statement.

## What to do if the Terminal is not running

Notice that I left a #take action here comment if the Terminal is not running. In this case we want to do something meaningful. This can be a simple notification telling the user to switch on the application, manually. But we are python samurais ⛩, our motto is: 

> _no mercy for manual entry_ 

and we are here to try to automatize the most our application, so we will make our machine open the terminal for us. The executable to launch the terminal is usually in ```C:\Program Files (x86)\Thomson Reuters\Eikon\Eikon.exe```. Python can lanch windows shell commands via the ```subprocess``` module or launch windows executable using the module ```os```. Either of the two following commands will launch the Terminal

In [1]:
# using os startfile method to launch the Eikon Terminal
import os
os.startfile("C:\Program Files (x86)\Thomson Reuters\Eikon\Eikon.exe")

# using the subprocess check_call method to launch the Eikon Terminal
import subprocess
subprocess.check_call([r"C:\Program Files (86)\Thomson Reuters\Eikon\Eikon.exe"])

## Wrap up everything in an auto-start tool

With all the information we have, we are now able to create a small tool that will try to open the terminal if it’s closed. In this way we are ‘triggerable’, meaning that we can launch our application when some event happens.

One example could be launching a script every morning, to fetch the closing prices of a security and feed a database.

Our application will still be robust enough to check if the terminal is opened without crashing. Moreover, we will not need to intervene manually to fix the problem, since our machine will solve it for us.

We can wrap what we’ve done in the following code:

In [ ]:
import eikon
import requests
import time
import os

# connect to the eikon api
eikon.set_app_key('your app key here')

def ping_eikon_proxy(max_attempts = 4):
    # counter for the number of attempts made
    attempts = 1
    # we start a loop until the attemps made are
    # > than the maximum number of attempts
    while attempts < max_attempts:
        try:
            # send a request to port 9060
            # returns a status 'ST_PROXY_READY' if eikon application is opened
            # throws a timeout error if the application is closed
            request = requests.get('http://localhost:9060/api/status')
            response = request.json()
            if response['statusCode'] == 'ST_PROXY_READY': 
                print(f'The Eikon proxy is up and running on port 9060')
                return True
        except:
            print(f'It seems your Eikon Terminal is not running. Attempt {attempts}/{max_attempts} to launch it.')
            # try to open the terminal using the executable
            os.startfile("C:\Program Files (x86)\Thomson Reuters\Eikon\Eikon.exe")
            # increase the attempt counter
            attempts +=1
            # wait 60 seconds before trying again. It takes some time to
            # open the terminal
            time.sleep(60)
    return False


ping_eikon_proxy()

let’s review the code above:

- we define a function called ```ping_eikon_proxy```, with input an integer representing the number of attempts to launch the terminal we want to make.

- We start while loop until the number of attempts exceeds the maximum attempts.

- We try to get a response with the method we saw earlier:

- If we succeed we exit with a ```True``` status.

- If not we try to open the terminal with ```os.startfile```, we increase the number of attempts and we wait 60 seconds before trying again the procedure.

- If after 4 attempts the terminal is not opened there is a major problem, and the while loop finishes, exiting the function with a ```False``` status.

Running the code above will produce the following output, if the terminal is closed:

```
It seems your Eikon Terminal is not running. Attempt 1/4 to launch it.
The Eikon proxy is up and running on port 9060
True
```

And here we are! You can now try to play a bit and write a cron/task job to retrieve some data each day, with the first check being the ```ping_eikon_proxy``` function. It will be a new tool in your Swiss Knife 🇨🇭 of development. So far we saw:

- How to retrieve live data
- How to retrieve time series data
- How to check if the terminal is up and running

These are all the blades composing your knife, but they are all separated. In the next Episodes we will create a nice case to old them all and to draw only the tools we need, when we need. Something portable that we can take everywhere and that will always behave the same way: a Python Class 🔧.